In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from pickle import load, dump
import warnings
import os
from random import sample
import imageio
import numpy as np

warnings.filterwarnings("ignore")

#path = "C:/Users/a.rojasa55/OneDrive - Universidad de los Andes/Documentos/MOPTA-23/Data/"
path = os.getcwd()
cd = 0.041
cw = 0.0388

In [2]:
stations = pd.read_csv(path+"/Data/Definite Stations.csv",index_col=[0])
vehicles = pd.read_csv(path+"/Data/Definite Vehicles.csv",index_col=[0])
locations = pd.read_csv(path+"/Data/Definite Locations.csv",index_col=[0])

In [3]:
file = open(path+"/Multiobjective/Configurations/Open Stations/open_stations_0.0","rb")
open_stats = load(file); file.close()

file = open(path+"/Multiobjective/Configurations/Stress Index/stress_index_0.0","rb")
stress_index = load(file); file.close()

file = open(path+"/Results/Optimal/S","rb")
S = load(file); file.close()

file = open(path+"/Results/Optimal/results","rb")
results = load(file); file.close()

file = open(path+"/Results/Optimal/n","rb")
n = load(file); file.close()

file = open(path+"/Results/Optimal/order","rb")
stats_order = load(file); file.close()


K = {}; S_k = {}
for sc in range(25):
    file = open(path+f"/Data/K/K_sc{sc}","rb")
    K[sc] = load(file); file.close()

    file = open(path+f"/Data/S_k10/Sk_sc{sc}","rb")
    S_k[sc] = load(file); file.close()


S_k = {sc:{k:[s for s in S_k[sc][k] if s in open_stats] for k in K[sc]} for sc in range(25)}
max_stress = max([stress_index[sc][s] for sc in range(sc) for s in open_stats]); min_stress = 0

In [4]:
def plot_solution(station,chargers,locations,stress_index,routes,prev_stats,sc=5):
    
    ''' Update stress indices and location bubble sizes '''
    if chargers > 0:
        for ssc in range(25):
            for i in routes[ssc][station][chargers-1]:
                for s in S_k[ssc][i]:
                    stress_index[ssc][s] -= 1
                if ssc == sc:
                    coord = i - int(i/1079)*1079
                    if coord == 0: coord = 1079
                    locations.loc[coord,f"going {sc}"] -= 1

    ''' Create plot '''
    img = plt.imread(path+"/Problem Visualization/Population grey.PNG")
    fig, ax = plt.subplots(nrows=1,ncols=1,figsize=(18,12))
    plt.ioff()
    
    ''' Vehicles '''
    locs = [l for l in locations.index if locations.loc[l,f"going {sc}"]>0]
    szs = [15+25*locations.loc[l,f"going {sc}"] for l in locs]
    ax.scatter(locations["0"][locs],locations["1"][locs],s=szs,color="navy",alpha=0.5)

    ''' Vehicle bubbles legend '''
    szs = {10:144.2, 9:138.4, 8:133, 7:127.8, 6:122.9, 5:118.5, 4:114.5, 3:110.8, 2:107.65, 1:105}
    for l in range(1,11):
        ax.scatter([-25],[szs[l]],marker="o",s=15+25*l,color="navy",alpha=0.5)
        if l in [1,10]:
            ax.text(x=-20,y=szs[l],s=l,va="center",ha="left",fontsize=14)
    ax.text(x=-32,y=126,rotation=90,s="vehicles per location", fontsize=14, va="center", ha="center", fontname="Century Gothic")
    
    ''' Color map legend '''
    cm = plt.get_cmap("gist_heat")
    for i in range(401):
        ax.bar(x=-25,height=0.1875,width=5,bottom=10.5+0.1875*i,color=cm((1-i/400)*0.9))
    for i in range(6):
        ax.text(x=-20,y=10.5+75*i/5,s=int((max_stress-min_stress)*i/5),va="center",ha="left",fontsize=14)
    ax.text(x=-32,y=48,rotation=90,s="stations stress index", fontsize = 14, va="center",ha="center", fontname="Century Gothic")
    
    ''' Stations Plot '''
    stats = [s for s in open_stats if (s not in prev_stats) and (chargers == 0 or (s != station))]
    cols = [cm((1-stress_index[sc][s]/max_stress)*0.9) for s in stats]
    ax.scatter(stations["0"][stats],stations["1"][stats],c=cols)

    ''' Previous Stations '''
    for s in prev_stats:
        ax.plot(stations.loc[s,"0"],stations.loc[s,"1"],marker="D",color="lime",markersize=10+1*(n[s]-1),alpha=0.7)

    ''' Current Station '''
    if chargers == 0:
        ax.plot(stations.loc[station,"0"],stations.loc[station,"1"],marker="D",markeredgewidth=2,markeredgecolor="lime",markerfacecolor="None",markersize=10)
    else:
        ax.plot(stations.loc[station,"0"],stations.loc[station,"1"],marker="D",color="lime",markersize=10+1*(chargers-1),alpha=0.7)
    
    ''' Current Station legend '''
    szs = {8:150.6, 7:141.8, 6:133.6, 5:125.8, 4:118.6, 3:111.9, 2:105.78, 1:100.1}
    for ch in range(1,9):
        if ch <= chargers: col = "lime"
        else: col = "grey"
        ax.plot(szs[ch],-17.5,marker="D",markersize=10+1*(ch-1),color=col,alpha=0.7)
    ax.text(x=125,y=-25,s="number of chargers",va="center",ha="center",fontsize=14,fontname="Century Gothic")
    if chargers == n[station]: col = "lime"
    else: col = "black"
    ax.text(x=70,y=-20,s=f"Current station\nID: {station}",va="center",ha="center",color=col,fontsize=16,fontname="Century Gothic")

    exp_stress = round(np.mean([stress_index[ssc][station] for ssc in range(25)]),2)
    if exp_stress == 0.0: col = "lime"
    elif chargers == n[station]: col = "red"
    else: col = "black"
    ax.text(x=160,y=-15,s=f"Expected Stress Index: {exp_stress}",fontsize=14,fontname="Century Gothic",va="center",ha="left",color=col)
    if stress_index[sc][station] == 0: col = "lime"
    elif chargers == n[station]: col = "red"
    else: col = "black"
    ax.text(x=160,y=-25,s=f"This scenario: {int(stress_index[sc][station])}",fontsize=14,fontname="Century Gothic",va="center",ha="left",color=col)

    ''' Map border '''
    ax.vlines(x=[0,290],ymin=0,ymax=150,color="black")
    ax.hlines(y=[0,150],xmin=0,xmax=290,color="black")
    ax.spines[["top","bottom","left","right"]].set_visible(False)

    ''' Ax config '''
    zero = (-5,-12)
    height = 184.425
    ax.imshow(img, extent=[zero[0], zero[0]+height*5/3, zero[1], zero[1]+height])
    
    ax.set_xlim(-40,320)
    ax.set_ylim(-30,170)
    ax.set_xticks([]); ax.set_yticks([])

    ''' Save plot '''
    fig.savefig(path+f"/Slides/SS_{station}_{chargers}.png",dpi=300,bbox_inches="tight")

    return locations, stress_index
    
    

In [5]:
for s in S:
    for sc in range(25):
        for i in range(n[s]-len(results[sc]["routes"][s])):
            results[sc]["routes"][s].append([])

In [ ]:
sc = 5
locs = locations[["0","1",f"going {sc}"]]
stress = stress_index.copy()
routes = {scc:results[scc]["routes"].copy() for scc in range(25)}

previous_stations = list()

for ix in range(1,41):
    station = stats_order[ix]
    for ch in range(n[station]+1):
        locs,stress = plot_solution(station=station,chargers=ch,locations = locs, stress_index=stress, routes = routes, prev_stats = previous_stations, sc = sc)
    previous_stations.append(station)

file = open(path+"/Slides/Partial/Partial1","wb")
dump((locs,stress,previous_stations),file); file.close()

In [ ]:
file = open(path+"/Slides/Partial/Partial1","rb")
locs,stress,previous_stations = load(file); file.close()
routes = {scc:results[scc]["routes"].copy() for scc in range(25)}; sc = 5

for ix in range(41,81):
    station = stats_order[ix]
    for ch in range(n[station]+1):
        locs,stress = plot_solution(station=station,chargers=ch,locations = locs, stress_index=stress, routes = routes, prev_stats = previous_stations, sc = sc)
    previous_stations.append(station)

file = open(path+"/Slides/Partial/Partial2","wb")
dump((locs,stress,previous_stations),file); file.close()

In [13]:
s = 802
print(f"Station {s}\t{n[s]} chargers\t{[len(results[sc]['routes'][s]) for sc in range(25)]}")

Station 802	6 chargers	[6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6]


In [16]:
for ssc in range(25):
    print(f"scenario {ssc} {routes[ssc][s][2-1]}")

scenario 0 [6469, 6474, 2154, 3232, 6470, 10786, 10784, 4313, 1078]
scenario 1 [5388, 6467, 10788, 5395, 9704, 3231, 6472, 1079, 10785]
scenario 2 [10789, 9704, 8631, 3231, 4314, 6474, 10790, 3232, 7550]
scenario 3 [1075, 5389, 3235, 5391, 7547, 7549, 3234, 2157, 7548, 2158, 7546, 9707]
scenario 4 [7546, 9710, 4310, 10787, 3233, 5390, 6471, 2158, 10788, 4316]
scenario 5 [10788, 2152, 1076, 8627, 7553, 6471, 9705, 9709, 9706, 4310, 10784, 5390]
scenario 6 [3235, 3236, 5394, 6470, 2152, 1073, 4309, 4316]
scenario 7 [7552, 10788, 1072, 8625, 4309, 9711, 3236, 7547]
scenario 8 [6467, 10788, 3233, 9705, 8630, 9709, 10784, 2155, 6472, 5390, 9706, 3236]
scenario 9 [6469, 4314, 6474, 5394, 5389, 3236, 3232, 4315, 9704, 10783, 8626, 2156]
scenario 10 [10790, 4314, 5391, 9709, 8630, 4313, 8629, 10785, 1075, 6468, 3231]
scenario 11 [3234, 3233, 2157, 5389, 6468, 9706, 6474, 2151, 4315, 1073]
scenario 12 [7547, 1078, 1075, 1077, 7549, 8630, 2151, 5389, 1079]
scenario 13 [8631, 5393, 1078, 10784, 8

In [ ]:
file = open(path+"/Slides/Partial/Partial2","rb")
locs,stress,previous_stations = load(file); file.close()
routes = {scc:results[scc]["routes"].copy() for scc in range(25)}; sc = 5

for ix in range(81,121):
    station = stats_order[ix]
    for ch in range(n[station]+1):
        locs,stress = plot_solution(station=station,chargers=ch,locations = locs, stress_index=stress, routes = routes, prev_stats = previous_stations, sc = sc)
    previous_stations.append(station)

file = open(path+"/Slides/Partial/Partial3","wb")
dump((locs,stress,previous_stations),file); file.close()

In [ ]:
file = open(path+"/Slides/Partial/Partial3","rb")
locs,stress,previous_stations = load(file); file.close()
routes = {scc:results[scc]["routes"].copy() for scc in range(25)}; sc = 5

for ix in range(121,155):
    station = stats_order[ix]
    for ch in range(n[station]+1):
        locs,stress = plot_solution(station=station,chargers=ch,locations = locs, stress_index=stress, routes = routes, prev_stats = previous_stations, sc = sc)
    previous_stations.append(station)

file = open(path+"/Slides/Partial/Partial4","wb")
dump((locs,stress,previous_stations),file); file.close()